In [54]:
import time
import requests
from bs4 import BeautifulSoup
import lxml
from tqdm.notebook import tqdm
import re
import json
from fake_useragent import UserAgent

import undetected_chromedriver 

import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date

import warnings
warnings.filterwarnings('ignore')

# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from seleniumwire import webdriver

import pickle

from selenium import webdriver
from selenium_stealth import stealth

In [ ]:
# указать пользователя
user = 'DANIL'
# если файл перенесен в другую папку то сменить путь ниже
path_to_credential = rf'C:/Users/{user}/Downloads/auto-monitoring-367212-64ec4ad9d3a5.json' 
executable_path = rf'C:\Users\{user}\personal_projects\selenuim_driver\chromedriver.exe'

In [ ]:
# Данные для доступа к Google Spreadsheets

# Specify path to your file with credentials
path_to_credential = path_to_credential 

# Указать название файла в google sheets
table_name = 'yandex_parser' 

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)

gs = gspread.authorize(credentials)
work_table = gs.open(table_name)


In [ ]:
# Получаем список ссылок из google sheets
worksheet = work_table.worksheet('gaming_console')
links_df = worksheet.get_all_values()
columns = links_df.pop(0)
links_df = pd.DataFrame(links_df, columns=columns)

In [ ]:
links_df

In [ ]:
print(links_df['Ссылка'][0])

In [61]:
def open_title_link(url):
    
    """ 
    функция вовзвращает объект soup по заданной ссылке на титульную страуницу товара
    
    url: str
        ссылка на титульную страницу товара
    """
    ua = UserAgent()
    user_agent = ua.random
    
    options = webdriver.ChromeOptions()
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(
        executable_path=executable_path,
        options=options)
    
    # for cookie in pickle.load(open("cookies_selenium", "rb")):
    #     driver.add_cookie(cookie)
    try:
        driver.get(url)
        time.sleep(2)
        pickle.dump(driver.get_cookies(), open("cookies_selenium", "wb"))
        soup = BeautifulSoup(driver.page_source, 'lxml')
    except Exception as ex:
        print(ex)
    finally:
        driver.close()
        driver.quit()
        
    return soup

In [62]:
open_title_link(links_df['Ссылка'][0])

<html prefix="og: http://ogp.me/ns#"><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title>Ой!</title><meta content="Яндекс" data-react-helmet="true" property="og:title"/><meta content="Найдётся всё" data-react-helmet="true" property="og:description"/><meta content="https://yastatic.net/s3/home/logos/share/share-logo-ru.png" data-react-helmet="true" property="og:image"/><link href="/captcha_smart.min.css?k=1632998364813" rel="stylesheet"/></head><body class="pointerfocus"><div id="root"><div class="Theme Theme_color_yandex-default Theme_root_default"><div class="Container"><div class="Spacer" style="padding-bottom:40px"><a class="Link Link_view_default LogoLink" href="https://www.yandex.ru" title="Яндекс"><svg height="36" viewbox="0 0 86 36" width="86"><path d="M45.983 28.888H44.385L44.377 11.578H35.027V13.274C35.027 18.556 34.835 24.67 32.997 28.888H31.758V35.313H34.555V31.41H43.186

In [ ]:
def get_info_title(soup):
    
    """ 
    функция вовзвращает информацию о товаре с его титульнойстраницы
    soup: BeautifulSoup object
        объект soup титульной страницы товара
    """
    
    # получасем кол-во человек, которые интересовались товаром за последние 2 месяца
    interest_badge_text = soup.find(attrs={'data-zone-name': "interest-badge"}).getText().replace(u'\xa0', u'')
    interest_badge_num = int(re.findall('\d+', interest_badge_text)[0])
    
    # получаем данные по рейтингу товара
    rating_dict = {}
    rating_meta = soup.find('div', attrs={'itemprop': "aggregateRating"}).find_all('meta')
    for item in rating_meta:
        rating_dict[item['itemprop']] = item['content']
    
    # получаем кол-во предложений и минимальная цена
    offers_minprice = soup.find('div', attrs={'data-zone-name': "morePricesLink"}).getText().replace(' ', '')
    offers, min_price = re.findall('\d+', offers_minprice)
    offers = int(offers)
    min_price = int(min_price)
    
    title_info_dict = {
        'интересовались за последние 2 месяца': interest_badge_num,
        'рейтинг': rating_dict, 
        'кол-во предложений': offers,
        'минимальная цена': min_price
    }
    
    return title_info_dict

In [ ]:
title_info = get_info_title(soup)
title_info

In [ ]:
def open_offers_link(url):
    
    """ 
    функция вовзвращает объект soup по заданной ссылке на страницу с предложениями по товару
    
    url: str
        ссылка на страницу с предложениями по товару
    """
    
    ua = UserAgent()
    user_agent = ua.random
    
    options = webdriver.ChromeOptions()
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(
        executable_path=f"C:/Users/{user}/Parsing_folder/chromedriver/chromedriver.exe",
        options=options)
    
    def get_offers_url(url):

        """функция возвращает ссылку на список предложений по товару
        url: str
             ссылка на страницу товара в Яндекс Маркете
        """

        splited_url = url.split('/')
        item_filter = splited_url.pop()
        splited_url.append('/offers?'.join(item_filter.split('?')))
        offers_url = '/'.join(splited_url) + '&how=aprice'

        return offers_url
    
    offers_url = get_offers_url(url)
    
    try:
        driver.get(offers_url)
        time.sleep(10)
        soup = BeautifulSoup(driver.page_source, 'lxml')
    except Exception as ex:
        print(ex)
    finally:
        driver.close()
        driver.quit()
        
    return soup

In [ ]:
def get_info_offers(soup):
    
    """ 
    функция вовзвращает информацию о предложениях товара
    
    soup: BeautifulSoup object
        объект soup страницы с предложениями товара
    """
    offers_info_dict = {}
    
    # получасем название магазина и цену
    snippet_list = soup.find('div', attrs={'data-zone-name': "snippetList"})
    
    for i in range(len(snippet_list)):
        offer_snippet = snippet_list.find_all('div', attrs={'data-zone-name': "OfferSnippet"})[i]\
            .find('div', attrs={'data-auto':'top-offer-snippet-shop-info'})
        
        if offer_snippet.find('div', attrs={'data-zone-name':'shop-name'}) is not None:
            shop_name_div = offer_snippet.find('div', attrs={'data-zone-name':'shop-name'})
            try:
                shop_name = shop_name_div.find('img')['title']
            except:
                shop_name = shop_name_div.find('a')['title']

        elif offer_snippet.find('a', attrs={'data-zone-name':'offerLink'}) is not None:
            shop_name = offer_snippet.find('a', attrs={'data-zone-name':'offerLink'}).getText()
        
        else:
            shop_name = 'unknown'
        
        price_snippet = snippet_list.find_all('div', attrs={'data-zone-name': "price"})[i]\
            .find('span', attrs={'data-auto':'mainPrice'})
        
        price = price_snippet.getText().replace(u'\xa0', u'').replace('₽', '').replace(' ', '')
        
        offers_info_dict[shop_name] = int(price)
    
    return offers_info_dict

In [ ]:
soup = open_offers_link(links_df['Ссылка'][0])

In [ ]:
get_info_offers(soup)